# Simple Sensor Checker

Given a station id and a date,
extracts the sensor




In [3]:
import os,sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import h5py as h5
import numpy as np
from tqdm import tqdm

In [15]:
# input data
station = '0-20000-0-03918'
station = '0-20000-0-82930'


merged_dir = '/mnt/users/scratch/leo/scratch/converted_v11/long/'
file = [merged_dir+'/'+f for f in os.listdir(merged_dir) if station in f and '.nc' in f and '.x' not in f and '.txt' not in f ][0]
file
f = h5.File(file, 'r')


In [6]:
f['era5fb'].keys()

<KeysViewHDF5 ['albedo@modsurf', 'an_depar@body', 'an_depar@surfbody_feedback', 'an_sens_obs@body', 'andate', 'antime', 'biascorr@body', 'biascorr_fg@body', 'bufrtype@hdr', 'class', 'codetype@hdr', 'collection_identifier@conv', 'date@hdr', 'datum_anflag@body', 'datum_event1@body', 'datum_rdbflag@body', 'datum_sfc_event@surfbody_feedback', 'datum_status@body', 'datum_status@surfbody_feedback', 'eda_spread@errstat', 'entryno@body', 'expver', 'fg_depar@body', 'fg_depar@surfbody_feedback', 'fg_error@errstat', 'final_obs_error@errstat', 'groupid@hdr', 'index', 'lat@hdr', 'lon@hdr', 'lsm@modsurf', 'lsm@surfbody_feedback', 'numtsl@desc', 'obs_error@errstat', 'obstype@hdr', 'obsvalue@body', 'orography@modsurf', 'ppcode@conv_body', 'qc_pge@body', 'report_event1@hdr', 'report_rdbflag@hdr', 'report_status@hdr', 'reportype', 'seaice@modsurf', 'sensor@hdr', 'seqno@hdr', 'snow_density@surfbody_feedback', 'snow_depth@modsurf', 'snow_depth@surfbody_feedback', 'sonde_type@conv', 'source@hdr', 'source_i

In [19]:
def extract_sensor(file, date):
    """ Read the file with h5py,
    concerts date in text format YYYYMMDD to pandas timestamp,
    extract sensor_id for the specified date """
    

    # converting date to timestamp
    dt = pd.Timestamp(date)
    
    # opening file with h5py
    f = h5.File(file, 'r')
    
    # timestamps from observations_table (contains the sensor_id )
    un = np.unique(f['observations_table']['date_time'])
    date_times = pd.to_datetime(un, unit='s',  origin=pd.Timestamp('1900-01-01') ).date 

    ind = np.where (date_times == dt )
    
    wmo = np.unique( [s for s in f['era5fb']['sonde_type@conv'][:][ind[0]] ] )
    sel = np.unique( [b''.join(s).decode('utf-8') for s in f['observations_table']['sensor_id'][:][ind[0]] ] )
    return wmo, sel 



    

In [13]:
# input data
merged_dir = '/mnt/users/scratch/leo/scratch/converted_v11/long/'
date = '19800601' 



#station = '0-20000-0-03918'
#file = [merged_dir+'/'+f for f in os.listdir(merged_dir) if station in f and '.nc' in f and '.x' not in f and '.txt' not in f ][0]
#file
#a = extract_sensor(file, date)



In [20]:
for station in tqdm([ '0-20000-0-03918','0-20000-0-06458', '0-20000-0-41112', '0-20000-0-71908', '0-20000-0-83779', '0-20000-0-47741' ]):
    file = [merged_dir+'/'+f for f in os.listdir(merged_dir) if station in f and '.nc' in f and '.x' not in f and '.txt' not in f ][0]
    date = '19800601'
    
    wmo,sel = extract_sensor(file, date)
    
    print(file , '  sensor list: ' , wmo , '  ', sel  )
    

 17%|█▋        | 1/6 [00:51<04:18, 51.60s/it]

/mnt/users/scratch/leo/scratch/converted_v11/long//0-20000-0-03918_CEUAS_merged_v1.nc   sensor list:  [nan nan nan nan]    ['VCL ']


 33%|███▎      | 2/6 [01:00<01:45, 26.43s/it]

/mnt/users/scratch/leo/scratch/converted_v11/long//0-20000-0-06458_CEUAS_merged_v1.nc   sensor list:  [nan nan nan nan]    ['VC_ ']


 50%|█████     | 3/6 [01:02<00:45, 15.22s/it]

...

/mnt/users/scratch/leo/scratch/converted_v11/long//0-20000-0-71908_CEUAS_merged_v1.nc   sensor list:  [nan nan]    ['VC_ ']


 83%|████████▎ | 5/6 [01:09<00:08,  8.27s/it]

/mnt/users/scratch/leo/scratch/converted_v11/long//0-20000-0-83779_CEUAS_merged_v1.nc   sensor list:  [nan nan]    ['VC_ ']


100%|██████████| 6/6 [02:06<00:00, 21.09s/it]

/mnt/users/scratch/leo/scratch/converted_v11/long//0-20000-0-47741_CEUAS_merged_v1.nc   sensor list:  [99.]    ['VN8 ']


In [ ]:
# Check From Schroeder original data


03918:  54.50:  -6.32:  20:VCL:200206302300:*+:Castor Bay, N Ireland, UK
03918:  54.50:  -6.32:  20:VFL:200302102300:*+:Castor Bay, N Ireland, UK

06458:  50.75:   4.77: 101:VC_:200411221400:*+:EBBE Beauvechain - Meteorological Wing, Base Charles Roman, Beauvechain, Belgium
06458:  50.75:   4.77: 101:DGM:200700000000:?-:EBBE Beauvechain - Meteorological Wing, Base Charles Roman, Beauvechain, Belgium

41112:  18.23:  42.65:2094:VC_:199909081200:*+:Abha, Saudi Arabia
41112:  18.23:  42.65:2094:Zng:200202091200:*+:Abha, Saudi Arabia
41112:  18.23:  42.65:2094:VC_:200202200000:*+:Abha, Saudi Arabia

71908:  53.9002:-122.7912: 601:VC_:199808140000:*+:Prince George - Environment Canada, 2400 Ospika Boulevard South, V2N 3N5 Prince George, Brit\

83779: -23.52: -46.63: 722:VC_:200105040000:*+:Sao Paulo - Marte Civilian/Military AP, Brazil

47741:  35.457: 133.066:  23:VN8:201003010000:*+:Matsue, Japan




